<a href="https://colab.research.google.com/github/SepehrBazyar/QDVRP/blob/master/Quality_Driven_Vehicle_Routing_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Dependencies**

In [1]:
!apt-get install -y -qq glpk-utils

In [2]:
!pip install -q condacolab

In [3]:
import condacolab


condacolab.install()

✨🍰✨ Everything looks OK!


In [4]:
!conda install pyomo cyipopt

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



**Import Libraries**

In [5]:
from abc import ABC
from dataclasses import dataclass, field

import pyomo.environ as pyo

**Define Model**

In [6]:
model = pyo.ConcreteModel()

**Parameters**

In [7]:
@dataclass(unsafe_hash=True)
class Base(ABC):
  id: int

In [8]:
@dataclass(unsafe_hash=True)
class Depot(Base):
  pass

In [9]:
@dataclass(unsafe_hash=True)
class Product(Base):
  beta: float
  quality: float
  temperature: int

In [10]:
@dataclass(unsafe_hash=True)
class Vehicle(Base):
  capacity: int
  salary: float
  alpha: float
  time_window: int
  increase_rate: float = 0.0  # degrees per minute

In [11]:
@dataclass(unsafe_hash=True)
class Customer(Base):
  unload_rate: float = 0.5  # in minutes

  total_demand: int = field(
    init=False,
    hash=False,
    default=0,
  )

  @property
  def service_time(self):
    return self.unload_rate * self.total_demand

In [12]:
temperature = 25

depot = Depot(id=0)
customers = (
  c1 := Customer(id=1),
  c2 := Customer(id=2),
  c3 := Customer(id=3),
  c4 := Customer(id=4),
  c5 := Customer(id=5),
  c6 := Customer(id=6),
  c7 := Customer(id=7),
)
nodes = depot, *customers

products = (
  p1 := Product(id=1, beta=0.01, quality=0.9, temperature=5),
  p2 := Product(id=2, beta=0.02, quality=0.85, temperature=3),
)

vehicles = (
  v1 := Vehicle(id=1, capacity=30, salary=1.5, alpha=5, time_window=180),
  v2 := Vehicle(id=2, capacity=25, salary=1.8, alpha=6, time_window=160),
  v3 := Vehicle(id=3, capacity=20, salary=2, alpha=7, time_window=150),
)

demands = {
  (p1, c1): 3, (p1, c2): 5, (p1, c3): 2, (p1, c4): 7, (p1, c5): 4, (p1, c6): 6, (p1, c7): 8,
  (p2, c1): 2, (p2, c2): 3, (p2, c3): 1, (p2, c4): 2, (p2, c5): 4, (p2, c6): 2, (p2, c7): 3,
}
for (_, customer), demand in demands.items():
  customer.total_demand += demand

distances = {
  (depot, c1): 10,
  (depot, c2): 15,
  (depot, c3): 20,
  (depot, c4): 25,
  (depot, c5): 30,
  (depot, c6): 22,
  (depot, c7): 27,
  (c1, c2): 12, (c1, c3): 18, (c1, c4): 22, (c1, c5): 28, (c1, c6): 16, (c1, c7): 21,
  (c2, c3): 14, (c2, c4): 20, (c2, c5): 24, (c2, c6): 18, (c2, c7): 26,
  (c3, c4): 16, (c3, c5): 21, (c3, c6): 24, (c3, c7): 30,
  (c4, c5): 12, (c4, c6): 15, (c4, c7): 19,
  (c5, c6): 19, (c5, c7): 17,
  (c6, c7): 14,
}
distances.update({(j, i): d for (i, j), d in distances.items()})

times = {
  (depot, c1): 20,
  (depot, c2): 25,
  (depot, c3): 30,
  (depot, c4): 35,
  (depot, c5): 40,
  (depot, c6): 45,
  (depot, c7): 50,
  (c1, c2): 15, (c1, c3): 20, (c1, c4): 25, (c1, c5): 30, (c1, c6): 35, (c1, c7): 40,
  (c2, c3): 10, (c2, c4): 15, (c2, c5): 20, (c2, c6): 25, (c2, c7): 30,
  (c3, c4): 10, (c3, c5): 15, (c3, c6): 20, (c3, c7): 25,
  (c4, c5): 10, (c4, c6): 15, (c4, c7): 20,
  (c5, c6): 10, (c5, c7): 15,
  (c6, c7): 10,
}
times.update({(j, i): t for (i, j), t in times.items()})

**Decision Varibales**

In [13]:
model.x = pyo.Var(nodes, nodes, vehicles, domain=pyo.Binary)
model.u = pyo.Var(customers, domain=pyo.NonNegativeReals)
# model.t = pyo.Var(vehicles, domain=pyo.NonNegativeReals)

In [14]:
v = {(j, k): sum(model.x[i, j, k] for i in nodes if i != j) for j in customers for k in vehicles}
q = {(p, k): sum(demands.get((p, j), 0) * v[j, k] for j in customers) for p in products for k in vehicles}
t = {
  k: (
    sum(times[i, j] * model.x[i, j, k] for i in nodes for j in nodes if i != j)
    +
    sum(j.service_time * v[j, k] for j in customers)
  ) for k in vehicles
}
d = {k: sum(j.service_time * k.increase_rate * v[j, k] for j in customers) for k in vehicles}

**Objective Function**

Minimize the Total Distance Traveled & Costs

In [15]:
def objective_func(model):
  distance_cost = sum(
    distances[i, j] * model.x[i, j, k]
    for i in nodes for j in nodes for k in vehicles if i != j
  )
  # cooling_cost = sum(
  #   k.alpha * t[k] * abs(temperature - (model.t[k] + d[k]))
  #   for k in vehicles
  # )
  # salary_cost = sum(k.salary * t[k] for k in vehicles)
  return distance_cost

In [16]:
model.obj = pyo.Objective(rule=objective_func, sense=pyo.minimize)

**Constraints**

Each Customer Visited Exactly Once by Exactly One Vehicle

In [17]:
def visit_once_rule(model, j: Customer):
  return sum(v[j, k] for k in vehicles) == 1

In [18]:
model.visit_once = pyo.Constraint(customers, rule=visit_once_rule)

Flow Conservation

If a Vehicle Enters a Node, It Must Leave It.

In [19]:
def flow_conservation_rule(model, j: Customer | Depot, k: Vehicle):
    come_in = sum(model.x[i, j, k] for i in nodes if i != j)
    outside = sum(model.x[j, i, k] for i in nodes if i != j)
    return come_in == outside

In [20]:
model.flow_conservation = pyo.Constraint(nodes, vehicles, rule=flow_conservation_rule)

Start & End at The Depot For Each Vehicle

In [21]:
def start_depot_rule(model, k: Vehicle):
    return sum(model.x[depot, i, k] for i in customers) <= 1


def end_depot_rule(model, k: Vehicle):
    return sum(model.x[i, depot, k] for i in customers) <= 1

In [22]:
model.start_depot = pyo.Constraint(vehicles, rule=start_depot_rule)
model.end_depot = pyo.Constraint(vehicles, rule=end_depot_rule)

Vehicle Capacity Constraints

In [23]:
def capacity_rule(model, k: Vehicle):
    return sum(q[p, k] for p in products) <= k.capacity

In [24]:
model.capacity = pyo.Constraint(vehicles, rule=capacity_rule)

Subtour Elimination

Miller-Tucker-Zemlin Formula

In [25]:
def subtour_elimination_rule(model, i: Customer, j: Customer, k: Vehicle):
    if i != j and i != depot and j != depot:
        return model.u[i] - model.u[j] + len(customers) * model.x[i, j, k] <= len(customers) - 1

    return pyo.Constraint.Skip

In [26]:
model.subtour_elimination = pyo.Constraint(customers, customers, vehicles, rule=subtour_elimination_rule)

Quality Maintaining

In [27]:
def quality_maintaining_rule(model, p: Product, k: Vehicle):
  return 1 - sum(p.beta * abs((model.t[k] + d[k]) - p.temperature) * v[j, k] for j in customers) >= p.quality

In [28]:
# model.quality_maintaining = pyo.Constraint(products, vehicles, rule=quality_maintaining_rule)

Time Window

In [29]:
def time_window_rule(model, k: Vehicle):
  return t[k] <= k.time_window

In [30]:
# model.time_window = pyo.Constraint(vehicles, rule=time_window_rule)

**Solve the Model**

In [31]:
solver_name = "glpk"

In [32]:
solver = pyo.SolverFactory(solver_name)

In [33]:
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 154.0, 'Upper bound': 154.0, 'Number of objectives': 1, 'Number of constraints': 166, 'Number of variables': 175, 'Number of nonzeros': 1050, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '4669', 'Number of created subproblems': '4669'}}, 'Error rc': 0, 'Time': 2.5587494373321533}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

**Display Results**

In [34]:
def get_path(router: dict[Customer, Customer], *, d: Depot = depot):
    order, next = [], router.get(depot, depot)
    while next != depot:
      order.append(str(next.id))
      next = router[next]

    if len(order) == 0:
      return "-"

    return " -> ".join((str(depot.id), *order, str(depot.id)))

In [35]:
for k in vehicles:
    router = {}
    for i in nodes:
        for j in nodes:
            if i != j and round(pyo.value(model.x[i, j, k])) == 1:
                router[i] = j

    t_k = "-"  # round(pyo.value(model.t[k]), ndigits=4)
    print(f"Vehicle {k.id} ({t_k}C): {get_path(router, d=depot)}")

Vehicle 1 (-C): 0 -> 3 -> 4 -> 5 -> 2 -> 0
Vehicle 2 (-C): 0 -> 6 -> 7 -> 1 -> 0
Vehicle 3 (-C): -
